In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
df = pd.read_csv('data.csv', header=None, names=['x1','x2','y'])
X, y = df[['x1','x2']].values, df['y'].values

## Part 1: Heuristic perceptron

In [ ]:
def train_perceptron(X, y, lr=0.1, epochs=20):
    w = np.random.randn(X.shape[1])
    b = 0.0
    history = [(w.copy(), b)]
    for _ in range(epochs):
        for xi, yi in zip(X, y):
            z = np.dot(w, xi) + b
            pred = 1 if z > 0 else 0
            error = yi - pred
            w += lr * error * xi
            b += lr * error
        history.append((w.copy(), b))
    return history


def plot_perceptron_grid(X, y, learning_rates, epochs_list):
    for epochs in epochs_list:
        fig = make_subplots(
            rows=1, cols=3,
            shared_xaxes=True, shared_yaxes=True,
            subplot_titles=[f"lr={lr}, epochs={epochs}" for lr in learning_rates]
        )

        for col, lr in enumerate(learning_rates, start=1):
            hist = train_perceptron(X, y, lr=lr, epochs=epochs)

            for lbl, color in zip([0,1], ['blue','orange']):
                mask = (y == lbl)
                fig.add_trace(
                    go.Scatter(
                        x=X[mask,0], y=X[mask,1],
                        mode='markers', marker=dict(size=6),
                        name=f"Class {lbl}",
                        showlegend=(col==1)
                    ),
                    row=1, col=col
                )

            for i, (w, b) in enumerate(hist):
                x0, x1 = 0.0, 1.0
                if abs(w[1]) > 1e-6:
                    y0 = -(w[0]*x0 + b)/w[1]
                    y1 = -(w[0]*x1 + b)/w[1]
                else:
                    x0 = x1 = -b / w[0]
                    y0, y1 = 0.0, 1.0

                if i == 0:
                    colr, dash, width, name = 'red',   'solid', 3, 'Initial'
                elif i == len(hist)-1:
                    colr, dash, width, name = 'black', 'solid', 3, 'Final'
                else:
                    colr, dash, width, name = 'green', 'dash',  2, None

                fig.add_trace(
                    go.Scatter(
                        x=[x0, x1], y=[y0, y1],
                        mode='lines',
                        line=dict(color=colr, dash=dash, width=width),
                        name=name,
                        showlegend=(name is not None and col==1)
                    ),
                    row=1, col=col
                )
        fig.update_layout(
            height=400, width=1200,
            title_text=f"Perceptron Boundaries – epochs={epochs}"
        )
        for c in [1,2,3]:
            fig.update_xaxes(title_text="x₁", range=[0,1], row=1, col=c)
            fig.update_yaxes(title_text="x₂", range=[0,1], row=1, col=c,
                             showticklabels=(c==1))

        fig.show()

learning_rates = [0.01, 0.1, 1.0]
epochs_list    = [20, 40, 80]
plot_perceptron_grid(X, y, learning_rates, epochs_list)


## Part 2: Logistic perceptron via gradient descent

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def train_logistic(X, y, lr=0.1, epochs=100):
    w = np.random.randn(X.shape[1])
    b = 0.0
    history = [(w.copy(), b)]
    losses = []
    for _ in range(epochs):
        for xi, yi in zip(X, y):
            z = np.dot(w, xi) + b
            y_hat = sigmoid(z)
            error = yi - y_hat
            w += lr * error * xi
            b += lr * error
        history.append((w.copy(), b))
        z_all = X.dot(w) + b
        y_hat_all = sigmoid(z_all)
        loss = -np.mean( y*np.log(y_hat_all+1e-9) + (1-y)*np.log(1-y_hat_all+1e-9) )
        losses.append(loss)
    return history, losses

def plot_logistic_grid(X, y, learning_rates, epochs_list):
    for epochs in epochs_list:
        fig = make_subplots(
            rows=2, cols=3,
            shared_xaxes=False,
            subplot_titles=(
                [f"Boundary: lr={lr}, ep={epochs}" for lr in learning_rates] +
                [f"Loss:     lr={lr}, ep={epochs}" for lr in learning_rates]
            )
        )

        for col, lr in enumerate(learning_rates, start=1):
            history, losses = train_logistic(X, y, lr=lr, epochs=epochs)
            for lbl, color in zip([0,1], ['blue','orange']):
                mask = (y == lbl)
                fig.add_trace(
                    go.Scatter(
                        x=X[mask,0], y=X[mask,1],
                        mode='markers', marker=dict(size=6),
                        name=f"Class {lbl}",
                        showlegend=(col==1)
                    ),
                    row=1, col=col
                )

            for i, (w, b) in enumerate(history):
                x0, x1 = 0.0, 1.0
                if abs(w[1]) > 1e-6:
                    y0 = -(w[0]*x0 + b)/w[1]
                    y1 = -(w[0]*x1 + b)/w[1]
                else:
                    x0 = x1 = -b / w[0]
                    y0, y1 = 0.0, 1.0

                if i == 0:
                    colr, dash, width, name = 'red',   'solid', 3, 'Initial'
                elif i == len(history)-1:
                    colr, dash, width, name = 'black', 'solid', 3, 'Final'
                else:
                    colr, dash, width, name = 'green', 'dash',  2, None

                fig.add_trace(
                    go.Scatter(
                        x=[x0, x1], y=[y0, y1],
                        mode='lines',
                        line=dict(color=colr, dash=dash, width=width),
                        name=name,
                        showlegend=(name is not None and col==1)
                    ),
                    row=1, col=col
                )

            fig.update_xaxes(range=[0,1], title_text="x₁", row=1, col=col)
            fig.update_yaxes(range=[0,1], title_text="x₂", row=1, col=col,
                             showticklabels=(col==1))
            fig.add_trace(
                go.Scatter(
                    x=list(range(1, len(losses)+1)),
                    y=losses,
                    mode='lines+markers',
                    name='Log Loss',
                    showlegend=False
                ),
                row=2, col=col
            )
            fig.update_xaxes(title_text="Epoch", row=2, col=col)
            fig.update_yaxes(title_text="Log Loss", row=2, col=col,
                             showticklabels=(col==1))

        fig.update_layout(
            height=800, width=1200,
            title_text=f"Logistic‑Regression Evolution – epochs={epochs}",
            legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
        )

        fig.show()

learning_rates = [0.01, 0.1, 1.0]
epochs_list    = [100, 150, 200]
plot_logistic_grid(X, y, learning_rates, epochs_list)